# Creating a final dataframe of phosphosite genomic locations and all relevent information.

In [1]:
#Import required packagaes
import pandas as pd

In [5]:
#Read in files to be merged.
kinase_df = pd.read_csv("new_clean_human_kinase_substrates.csv")  #Kinase/substarte dataframe.
Biomart_df = pd.read_csv("Biomart_sub_gene_locations.csv")  #Genomic locations of substrate genes from ensembl biomart.
PS_gl_df = pd.read_csv("PS_genomic_locations.csv") #Genomic coordinates of individual phosphosites.


In [6]:
#From kinase_df create new dataframe only with relevant columns.
kinase_sub_rsd_df=kinase_df[['KINASE', 'KIN_ACC_ID', 'SUB_ACC_ID', 'SUB_GENE', 'SUB_MOD_RSD', 'SITE_+/-7_AA']]


In [7]:
#Change Biomart 'gene name' column to 'SUB_GENE'to simplify the joining of dataframes.
Biomart_df=Biomart_df.rename(columns={"Gene name": "SUB_GENE"})

#join genomic locations from Biomart with relevant columns from kinase/substrate dataframe.
#This can be joined because each substrate gene has a single genomic location, although can have multiple phosphosites and kinases.
Biomart_and_kinase=kinase_sub_rsd_df.join(Biomart_df.set_index('SUB_GENE'), on='SUB_GENE')


In [8]:
#Change PS_gl_df 'PS' column to 'SUB_MOD_RSD' to simplify merging of dataframes.
PS_gl_df=PS_gl_df.rename(columns={"PS": "SUB_MOD_RSD"})


##Merge phosphosite genomic coordinates with rest of the relevant information.
#This has to be merged as the same substrate will have multiple phosphosites which have different coordiantes, so merge by 2 columns ('SUB_ACC_ID' and 'SUB_MOD_RSD'). 
Phosphosite_genomic_locations=pd.merge(Biomart_and_kinase, PS_gl_df, on=['SUB_ACC_ID', 'SUB_MOD_RSD'])

#Create final csv file from resulting dataframe.
Phosphosite_genomic_locations.to_csv('Phosphosite_genomic_locations.csv', sep=',', header=True, index=False) 

# Obtaining genomic coordinates of the substarte genes and producing corresponding url to connect to UCSC genome browser.

In [2]:
#Read in df with start and end coordinates for whole gene of the substrate.
#This data obtained from ensembl biomart using file withsubstrate accession numbers as before.
sub_gene_start_end=pd.read_csv("sub_gene_start_end.csv")

In [3]:
#make corresponding url for each substrate gene to be used in genome browser for website.

empty_url = "https://genome.ucsc.edu/cgi-bin/hgTracks?db=hg38&lastVirtModeType=default&lastVirtModeExtraState=&virtModeType=default&virtMode=0&nonVirtPosition=&position=chr{}%3A{}%2D{}&hgsid=796473843_RdusyHlWn1O3a5PrtgCz1VDHBQGv"

ch=sub_gene_start_end['Chromosome/scaffold name']
start =sub_gene_start_end['Gene start (bp)']
end=sub_gene_start_end['Gene end (bp)']

   
url_list=[]
for c,s,e in zip(ch, start, end):
    full_url=empty_url.format(c,s,e)
    url_list.append(full_url)


In [4]:
#Put substrate gene names in a list format.
sub_gene_list = sub_gene_start_end['Gene name'].tolist()

#Create and fill a dataframe of substrate gene names and urls to go in.
sub_gene_urls_df=pd.DataFrame()
sub_gene_urls_df=sub_gene_urls_df.append(pd.DataFrame(list(zip(sub_gene_list, url_list)), columns =['Gene name', 'URL']))

#Output file with urls to csv.
sub_gene_urls_df.to_csv('sub_gene_urls.csv', sep=',', header=True, index=False)

In [23]:
#Rename 'Gene name' to 'SUB_GENE' to allowfor a simple join command.
sub_gene_urls_df=sub_gene_urls_df.rename(columns={"Gene name": "SUB_GENE"})

#Join a column containing urls based on the substrate gene name ('SUB_GENE').
Phosphosite_genomic_locations_and_urls=Phosphosite_genomic_locations.join(sub_gene_urls_df.set_index('SUB_GENE'), on='SUB_GENE')

#output to csv
Phosphosite_genomic_locations_and_urls.to_csv('Final_Phosphosite_genomic_locations.csv', sep=',', header=True, index=False)

# Checking the urls

In [3]:
#loading the new file with urls
url_df = pd.read_csv("Final_Phosphosite_genomic_locations.csv")
url_df[url_df["SUB_GENE"]=="EIF2S1"]

,KINASE,KIN_ACC_ID,SUB_ACC_ID,SUB_GENE,SUB_MOD_RSD,SITE_+/-7_AA,Chromosome/scaffold name,Karyotype band,Strand,Start co,End co,URL
0,HRI,Q9BQI3,P05198,EIF2S1,S49,IEGMILLsELsRRRI,14,q23.3,1.0,67364912,67364914,https://genome.ucsc.edu/cgi-bin/hgTracks?db=hg...
1,PKR,P19525,P05198,EIF2S1,S49,IEGMILLsELsRRRI,14,q23.3,1.0,67364912,67364914,https://genome.ucsc.edu/cgi-bin/hgTracks?db=hg...
2,PERK,Q9NZJ5,P05198,EIF2S1,S49,IEGMILLsELsRRRI,14,q23.3,1.0,67364912,67364914,https://genome.ucsc.edu/cgi-bin/hgTracks?db=hg...


In [5]:
url_df.iloc[0]["URL"] == url_df.iloc[1]["URL"]

True

This shows that the urls are duplicated as they are substrates specific.